In [1]:
import pandas as pd

In [2]:
year = 2021
election = "federal"

# Canada

2021: https://www.elections.ca/content.aspx?section=res&dir=cir/list&document=index338&lang=e

In [3]:
# https://www.elections.ca/content.aspx?section=res&dir=cir/list&document=index338&lang=e
districts_2021 = {
    "10": "Newfoundland and Labrador",
    "11": "Prince Edward Island",
    "12": "Nova Scotia",
    "13": "New Brunswick",
    "24": "Quebec",
    "35": "Ontario",
    "46": "Manitoba",
    "47": "Saskatchewan",
    "48": "Alberta",
    "59": "British Columbia",
    "60": "Yukon",
    "61": "Northwest Territories",
    "62": "Nunavut"
}

def create_polling_id(data, columns=[]):
    output = data[columns[0]].fillna("###").astype(str).copy()

    for column in columns[1:]:
        output += "-" + data[column].fillna("###").astype(str)
    return output

In [18]:
df = pd.read_csv(f"../data/Canada/{year}_first_round.csv")
# df["candidate"] = df.apply(lambda x: f'{x["Candidate’s First Name/Prénom du candidat"]} {x["Candidate’s Family Name/Nom de famille du candidat"]}', axis=1)
df = df.rename(columns={
    "Candidate Poll Votes Count/Votes du candidat pour le bureau": "value",
    "Electoral District Number/Numéro de circonscription": "district_id",
    "Political Affiliation Name_English/Appartenance politique_Anglais": "party",
    "Electoral District Name_English/Nom de circonscription_Anglais": "district",
    "Electoral District Name_French/Nom de circonscription_Français": "district_fr"
})
df["polling_id"] = create_polling_id(
    df, 
    columns=["district_id", "Polling Station Number/Numéro du bureau de scrutin", "Polling Station Name/Nom du bureau de scrutin"])
df.head()

,district_id,district,district_fr,Polling Station Number/Numéro du bureau de scrutin,Polling Station Name/Nom du bureau de scrutin,Void Poll Indicator/Indicateur de bureau supprimé,No Poll Held Indicator/Indicateur de bureau sans scrutin,Merge With/Fusionné avec,Rejected Ballots for Polling Station/Bulletins rejetés du bureau,Electors for Polling Station/Électeurs du bureau,Candidate’s Family Name/Nom de famille du candidat,Candidate’s Middle Name/Second prénom du candidat,Candidate’s First Name/Prénom du candidat,party,Political Affiliation Name_French/Appartenance politique_Français,Incumbent Indicator/Indicateur_Candidat sortant,Elected Candidate Indicator/Indicateur du candidat élu,value,polling_id
0,10001,Avalon,Avalon,1,Freshwater,N,N,NaN,2,106,Chapman,NaN,Matthew,Conservative,Conservateur,N,N,28,10001- 1-Freshwater
1,10001,Avalon,Avalon,1,Freshwater,N,N,NaN,2,106,Davis,NaN,Carolyn,NDP-New Democratic Party,NPD-Nouveau Parti démocratique,N,N,13,10001- 1-Freshwater
2,10001,Avalon,Avalon,1,Freshwater,N,N,NaN,2,106,McDonald,NaN,Ken,Liberal,Libéral,N,Y,55,10001- 1-Freshwater
3,10001,Avalon,Avalon,1,Freshwater,N,N,NaN,2,106,Stewart,NaN,Lainie,People's Party - PPC,Parti populaire - PPC,N,N,4,10001- 1-Freshwater
4,10001,Avalon,Avalon,2,Victoria,N,N,NaN,0,330,Chapman,NaN,Matthew,Conservative,Conservateur,N,N,56,10001- 2-Victoria


In [12]:
df["candidate"] = df["Candidate’s First Name/Prénom du candidat"] + " " + df["Candidate’s Family Name/Nom de famille du candidat"]

In [13]:
df.head()

,district_id,district,district_fr,Polling Station Number/Numéro du bureau de scrutin,Polling Station Name/Nom du bureau de scrutin,Void Poll Indicator/Indicateur de bureau supprimé,No Poll Held Indicator/Indicateur de bureau sans scrutin,Merge With/Fusionné avec,Rejected Ballots for Polling Station/Bulletins rejetés du bureau,Electors for Polling Station/Électeurs du bureau,Candidate’s Family Name/Nom de famille du candidat,Candidate’s Middle Name/Second prénom du candidat,Candidate’s First Name/Prénom du candidat,party,Political Affiliation Name_French/Appartenance politique_Français,Incumbent Indicator/Indicateur_Candidat sortant,Elected Candidate Indicator/Indicateur du candidat élu,value,polling_id,candidate
0,10001,Avalon,Avalon,1,Freshwater,N,N,NaN,2,106,Chapman,NaN,Matthew,Conservative,Conservateur,N,N,28,10001- 1-Freshwater,Matthew Chapman
1,10001,Avalon,Avalon,1,Freshwater,N,N,NaN,2,106,Davis,NaN,Carolyn,NDP-New Democratic Party,NPD-Nouveau Parti démocratique,N,N,13,10001- 1-Freshwater,Carolyn Davis
2,10001,Avalon,Avalon,1,Freshwater,N,N,NaN,2,106,McDonald,NaN,Ken,Liberal,Libéral,N,Y,55,10001- 1-Freshwater,Ken McDonald
3,10001,Avalon,Avalon,1,Freshwater,N,N,NaN,2,106,Stewart,NaN,Lainie,People's Party - PPC,Parti populaire - PPC,N,N,4,10001- 1-Freshwater,Lainie Stewart
4,10001,Avalon,Avalon,2,Victoria,N,N,NaN,0,330,Chapman,NaN,Matthew,Conservative,Conservateur,N,N,56,10001- 2-Victoria,Matthew Chapman


In [14]:
tt = df.groupby(["polling_id", "candidate"]).agg({"value": "sum"})
tt["rate"] = tt.groupby(level=[0], group_keys=False).apply(lambda x: x/x.sum())
tt = tt.reset_index()

tt["rank"] = tt.groupby("polling_id")["value"].rank("min", ascending=False).astype(int)

tt

,polling_id,candidate,value,rate,rank
0,10001- 1-Freshwater,Carolyn Davis,13,0.130000,3
1,10001- 1-Freshwater,Ken McDonald,55,0.550000,1
2,10001- 1-Freshwater,Lainie Stewart,4,0.040000,4
3,10001- 1-Freshwater,Matthew Chapman,28,0.280000,2
4,10001- 10-Carbonear,Carolyn Davis,6,0.069767,3
...,...,...,...,...,...
477528,62001- S/R 1-SVR Group 1/RÉS Groupe 1,Lori Idlout,83,0.393365,2
477529,62001- S/R 1-SVR Group 1/RÉS Groupe 1,Pat Angnakak,94,0.445498,1
477530,62001- S/R 2-SVR Group 2/RÉS Groupe 2,Laura Mackenzie,66,0.168798,3
477531,62001- S/R 2-SVR Group 2/RÉS Groupe 2,Lori Idlout,184,0.470588,1


In [19]:
tt.groupby("candidate").agg({"value": "sum"})

,value
candidate,
Abdifatah Abdi,3060
Above Znoneofthe,418
Adam Chambers,27383
Adam DeVita,3995
Adam Kilner,11990
...,...
Ziad Aboultaif,20219
Ève Péclet,9579
Élisabeth Brière,21830


In [16]:
data = pd.merge(tt, df[["party", "candidate"]].drop_duplicates(), on="candidate")
data.head()

,polling_id,candidate,value,rate,rank,party
0,10001- 1-Freshwater,Carolyn Davis,13,0.130000,3,NDP-New Democratic Party
1,10001- 10-Carbonear,Carolyn Davis,6,0.069767,3,NDP-New Democratic Party
2,10001- 100-Conception Bay South,Carolyn Davis,19,0.223529,3,NDP-New Democratic Party
3,10001- 101-Conception Bay South,Carolyn Davis,50,0.273224,3,NDP-New Democratic Party
4,10001- 102-Conception Bay South,Carolyn Davis,42,0.234637,3,NDP-New Democratic Party


In [17]:
data.to_csv(f"../data_output/Canada/{year}_{election}.csv.gz", compression="gzip", index=False)

In [8]:
df_location = df[["polling_id", "district_id", "district", "district_fr"]].drop_duplicates()
df_location["state_id"] = df_location["district_id"].astype(str).str[0:2].copy()
df_location["state"] = df_location["state_id"].replace(districts_2021)
df_location.to_csv(f"../data_output/Canada/{year}_{election}_location.csv.gz", compression="gzip", index=False)
df_location.head()

,polling_id,district_id,district,district_fr,state_id,state
0,10001- 1-Freshwater,10001,Avalon,Avalon,10,Newfoundland and Labrador
4,10001- 2-Victoria,10001,Avalon,Avalon,10,Newfoundland and Labrador
8,10001- 3-Victoria,10001,Avalon,Avalon,10,Newfoundland and Labrador
12,10001- 4-Victoria,10001,Avalon,Avalon,10,Newfoundland and Labrador
16,10001- 5-Victoria,10001,Avalon,Avalon,10,Newfoundland and Labrador
